Import the necessary **libraries**

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Model summary
\begin{array}{ccc}
\text{Layer}&\text{Input}&\text{Output}&\text{Receptive Field}\\
Conv1&28*28*1&28*28*32&3*3\\
Conv2&28*28*32&28*28*64&5*5\\
Pool1&28*28*64&14*14*64&10*10\\
Conv3&14*14*64&14*14*128&12*12\\
Conv4&14*14*128&14*14*256&14*14\\
Pool2&14*14*256&7*7*256&28*28\\
Conv5&7*7*256&5*5*512&30*30\\
Conv6&5*5*512&3*3*1024&32*32\\
Conv7&3*3*1024&1*1*10&34*34\\
\end{array}


## Optimized model summary
\begin{array}{ccc}
\text{Layer}&\text{Input}&\text{Output}&\text{Receptive Field}\\
Conv1&28*28*1&28*28*32&3*3\\
Conv2&28*28*32&28*28*64&5*5\\
Pool1&28*28*64&14*14*64&10*10\\
Conv3&14*14*64&14*14*128&12*12\\
Conv4&14*14*128&14*14*256&14*14\\
Pool2&14*14*256&7*7*256&28*28\\
Conv5&7*7*256&5*5*512&30*30\\
Conv6&5*5*512&3*3*1024&32*32\\
fc1&3*3*1024&1*1*10&--\\
\end{array}

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) 
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)
        self.fc1 = nn.Linear(9216, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        
        #x = F.relu(self.conv7(x))
        x = torch.flatten(x, 1)
        #print("before 1 ", x.shape)
        x = self.fc1(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

## summary

\begin{array}{ccc}
\text{Layer}&\text{calculation}&\text{No of parameters}\\
Conv1&3*3*32 + 32(bias)&320\\
Conv2&3*3*32*64 + 64(bias)&18,496\\
Conv3&3*3*64*128 + 128(bias)&73,856\\
Conv4&3*3*128*256 + 256(bias)&295,168\\
Conv5&3*3*256*512 + 512(bias)&1,180,160\\
Conv6&3*3*512*1024 + 1024(bias)&4,719,616\\
Conv7&3*3*1024*10 + 10(bias)&92,170\\
\end{array}

In [ ]:
!pip install torchsummary
from torchsummary import summary

# Check if gpu is available 
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# direct the model to run in GPU
model = Net().to(device)
# give a summary of the model like output of each layer , number of parameters in each layer
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Linear-9                   [-1, 10]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


### Pre-processing



*   Take a batch size of 128
*   Call the DataLoader api 
*   > use the MNIST dataset from pytorch
*   > normalize the data - mean = 0.1307 , std = 0.3081
*   > shuffle the data 






In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


### Function to train 

*   model -> set the mode to train 
*   tqdm is used to show progress bar while looping through the train/test dataset
*   while looping through each batch of train data and label
*   set the device 
*   set the gradient to zero before doing backpropogation , because pytorch accumulates the gradient on subsequent backward passes. 
*   train the model on each batch 
*   get the negative log likelihood loss between the predicted label and the true label
*   backpropogate the loss to readjust the parameter weights

### Function to test 


*   model -> set the mode to eval 
*   disable gradient calculation
*   while looping through each batch of test data and label
*   set the device 
*   test the model on each batch 
*   get the negative log likelihood loss between the predicted label and the true label 
*   sum the batch loss
*   find the number of correct prediction
*   measure the accuracy of the model = correct / total number of records




In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### run the model on the train data and evaluate the model on test data

*   Use stochastic gradient descent to backpropogate and upddate the parameters
*   for each epoch , train the model and evaluate it performance on test data



In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 5):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0634, Accuracy: 9798/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0544, Accuracy: 9833/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0398, Accuracy: 9883/10000 (99%)



loss=0.01566830836236477 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 35.41it/s]



Test set: Average loss: 0.0284, Accuracy: 9909/10000 (99%)

